In [1]:
import os

os.chdir("..")

In [2]:
import torch
from diffusers import FluxKontextPipeline
from PIL import Image
import helpers.drawing as drawing
import numpy as np
import cv2
import shutil


In [3]:
# !export HF_HUB_CACHE=./cache

## Normal FluxKontext Generation

In [5]:
pipe = FluxKontextPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-Kontext-dev", torch_dtype=torch.bfloat16
)
pipe = pipe.to("cuda")


Loading pipeline components...: 100%|█████████████████████████████████████████| 7/7 [00:00<00:00,  7.14it/s]


## Extract the subject/character

In [6]:

def generate_image(base_scene_path, save_path, prompt = "Add character to the image.", width=None, height=None):
    base_scene = Image.open(base_scene_path).convert("RGB")

    if width is None:
        width, height = base_scene.size

    seed = torch.Generator().manual_seed(42)

    result_img_base = pipe(
        prompt=prompt,
        image=base_scene,
        num_inference_steps=28,
        height=height,
        width=width,
        # generator=seed,
        _auto_resize=False
    ).images[0]
    result_img_base.save(save_path)
    return result_img_base

100%|███████████████████████████████████████████████████████████████████████| 14/14 [00:17<00:00,  1.23s/it]


In [21]:
src = "custom_data/product"

end_dir = os.path.join(src, "original")
image_extensions = (".png", ".jpg", ".jpeg", ".webp")
image_paths = [
    os.path.join(end_dir, fname)
    for fname in os.listdir(end_dir)
    if fname.lower().endswith(image_extensions)
]
image_paths.sort()

# Create end, reference and start folders, if it doesnot exists
os.makedirs(os.path.join(src, "end"), exist_ok=True)
os.makedirs(os.path.join(src, "reference"), exist_ok=True)
os.makedirs(os.path.join(src, "start"), exist_ok=True)

len(image_paths)

104

In [4]:

# Rename all images in end_dir as 000.<ext>, 001.<ext>, etc.
for idx, old_path in enumerate(sorted(image_paths)):
    ext = os.path.splitext(old_path)[1].lower()
    new_name = f"{idx:03d}{ext}"
    new_path = os.path.join(end_dir, new_name)
    shutil.move(old_path, new_path)

# Update image_paths to reflect new names
image_paths = [
    os.path.join(end_dir, fname)
    for fname in sorted(os.listdir(end_dir))
    if fname.lower().endswith(image_extensions)
]

In [28]:

i = 11

In [121]:
i = i+1
drawing.crop_image(image_paths[i], image_paths[i].replace("original", "end"))


IndexError: list index out of range

In [20]:
end_dir = os.path.join(src, "end")
image_extensions = (".png", ".jpg", ".jpeg", ".webp")
image_paths = [
    os.path.join(end_dir, fname)
    for fname in os.listdir(end_dir)
    if fname.lower().endswith(image_extensions)
]



In [ ]:
reference_prompt = "Extract the boy from the image, in a white background."
for image_path in image_paths:
    reference_save_path = image_path.replace("end", "reference")
    generate_image(image_path, reference_save_path, reference_prompt, 512, 512)

In [ ]:
start_prompt = "Remove the boy from the image."
for image_path in image_paths:
    start_save_path = image_path.replace("end", "start")
    generate_image(image_path, start_save_path, start_prompt)